# Higher-Order Models

[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/higher_order_models.ipynb)  
[Download notebook](https://github.com/pathpy/pathpy/raw/master/doc/tutorial/higher_order_models.ipynb)

In this notebook, we show you the basics of working with higher-order network models in `pathpy`.
To start, we work in a simple example of paths. Rather than calculating causal paths based on a time-stamped network data set, we take a simpler approach and directly specify the causal paths that we want to model. For this, we can use the `Paths` class in `pathpy`. We can use it to store paths of the form `a,b,c,d,...` of arbitrary length. This object will automatically calculate the statistics needed to fit higher-order models, as well as select the optimal models' order.


In [ ]:
pip install git+git://github.com/pathpy/pathpy.git

In [ ]:
import pathpy as pp
import random
from matplotlib import pyplot as plt
from matplotlib.colors import to_hex
from matplotlib import cm
import numpy as np
from collections import defaultdict
from IPython.display import clear_output

plt.xkcd()

style = {
    'width': 700,
    'height': 600,
    'edge_arrows': False,
    'plot_higher_order_nodes': True,
    'd3js_path': 'http://localhost:8888/notebooks/d3.v4.min.js',
    'max_time': 500,
}



We start by creating an empty instance of `Paths` and adding two causal paths `a->b->c` and `b->c->e`. We print the resulting object and visualise the underlying network by constructing a `Network` instance using the class method `Network.from_paths`.

In [ ]:
paths_1 = pp.Paths()
paths_1.add_path("a,c,d", 2)
paths_1.add_path("b,c,e", 2)
print(paths_1)

n = pp.Network.from_paths(paths_1)
pp.visualisation.plot(n, **style)

The network representation above decomposes the paths into individual links, i.e. paths of length one that are represented by the network topology. Based on this network topology, we would expect that node `a` can reach `e` via node `c`, which is however not possible based on the observed paths. We can mitigate this by generalising the (first-order) network model above to higher-order network models of order $k$.

`pathpy` provides the class `HigherOrderNetwork`, which we can use to fit a higher-order model with a given order `k` to the data.

In [ ]:
hon_1 = pp.HigherOrderNetwork(paths_1, k=1)
pp.visualisation.plot(hon_1, **style)

In [ ]:
print(hon_1)
for e in hon_1.edges:
    print('{0} {1}'.format(e, hon_1.edges[e]['weight'].sum()))

In [ ]:
hon_2 = pp.HigherOrderNetwork(paths_1, k=2)
pp.visualisation.plot(hon_2, **style)

In [ ]:
print(hon_2)
for e in hon_2.edges:
    print('{0} {1}'.format(e, hon_2.edges[e]['weight'].sum()))

## Higher-Order Model Inference and Selection

We can interpret different higher-order models as **probabilistic generative models** for the paths that we have observed. Conveniently, the `HigherOrderNetwork` class in `pathpy` comes with a method `likelihood`. It calculates the probability to observe exactly the paths stored in a given `Paths` instance, given a higher-order network model. This allows us to calculate the likelihood of the higher-order model given the observed paths.

To show how the method works, we again start with a maximaly simple toy example:

In the example `paths_1` we only observe two of the four paths of length two that would be possible in the null model. Hence, this is an example for path statistics that exhibit correlations that warrant a second-order model.

But how can we decide this in a meaningful way? We can take a statistical inference perspective on the problem. More specifically, we will consider our higher-order networks as probabilistic generative models for paths in a given network topology. For this, let us use the weighted first-order network model to construct a `transition matrix` of a Markov chain model for paths in a network. We simply use the relative frequencies of edges to proportionally scale the probabilities of edge transitions in the model.

In [ ]:
hon_1 = pp.HigherOrderNetwork(paths_1)
pp.visualisation.plot(hon_1)
print(hon_1.transition_matrix().todense())

This transition matrix defines a first-order Markov chain model for paths in the underlying network topology. This probabilistic view allows us to calculate a likelihood of the first-order model, given the paths that we have observed. With `pathpy`, we can directly calculate the `likelihood` of a higher-order model, given a Paths instance.

In [ ]:
print(hon_1.likelihood(paths_1, log=False))

This result is easy to understand for our toy example. Each path of length two corresponds to two transitions in the transition matrix of our Markov chain model. For each of the four paths of length two in toy_paths, the first transition is deterministic because nodes a and b only point to node c. However, based on the network topology, for the second step we have a choice between nodes d and e. Considering that we see as many transitions through edge (c,d) as we see through edge (c,e) , in a first-order model we have no reason to prefer one over the other, so each is assigned probability 0.5.

Hence, for each of the four observed paths we obtain a likelihood of $1 \cdot 0.5=0.5$, which yields a total likelihood for four (independent) observations of $0.5^4 = 0.0625$.

Let us compare this to the likelihood of a second-order model for our paths.

In [ ]:
hon_2 = pp.HigherOrderNetwork(paths_1, k=2)
pp.visualisation.plot(hon_2)
print(hon_2.transition_matrix().todense())
hon_2.likelihood(paths_1, log=False)

In [ ]:
Here, the likelihood is 1, because all transitions in the second-order model are deterministic, i.e. we multiply 1⋅1 four times.

Let us now have a look at the second-order null model, which is actually a first-order model represented in the second-order space. So we should expect the same likelihood as the first-order model.

In [ ]:
hon_2_null = pp.HigherOrderNetwork(paths_1, k=2, null_model=True)
pp.visualisation.plot(hon_2_null)

print(hon_2_null.transition_matrix().todense())
hon_2_null.likelihood(paths_1, log=False)